In [96]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [98]:
from py2neo import Graph
from transformers import AutoTokenizer, AutoModel
import torch
import faiss
import numpy as np

# Kết nối với Neo4j và lấy dữ liệu môn học từ ontology
neo4j_graph = Graph("bolt://localhost:7687", auth=("neo4j", "123456789"), name="edu-db")

# Truy vấn để lấy thông tin các môn học
query = """
  MATCH (ancestor:Resource {uri: 'http://localhost/ontologies/2024/10/11/edu_program#monHoc'})
  MATCH (n:Resource)-[:rdfs__subClassOf*]->(ancestor)
  MATCH (instance:Resource)-[:rdf__type]->(n)
  RETURN DISTINCT instance.ns0__maMonHoc AS code, instance.rdfs__label AS label
"""
results = neo4j_graph.run(query).data()

# Danh sách môn học và tên
subjects = [(result['code'], result['label']) for result in results]
labels = [subject[1] for subject in subjects if subject[1]]

# Từ điển từ viết tắt và từ đồng nghĩa
abbreviation_dict = {
    "HTTT": "hệ thống thông tin",
    "CNPM": "công nghệ phần mềm",
    "ATTT": "an toàn thông tin",
    "CNTT": "công nghệ thông tin",
    "AI": "trí tuệ nhân tạo",
    "CSDL": "cơ sở dữ liệu"
}

# Khởi tạo PhoBERT
phobert_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
phobert_model = AutoModel.from_pretrained("vinai/phobert-base")

def encode_sentence(sentence, tokenizer, model):
    """
    Mã hóa câu sử dụng PhoBERT và trả về vector ngữ nghĩa
    """
    # Thay thế từ viết tắt bằng nghĩa đầy đủ
    for abbr, full_form in abbreviation_dict.items():
        sentence = sentence.replace(abbr, full_form)
    
    tokens = tokenizer.encode(sentence, return_tensors='pt')
    with torch.no_grad():
        output = model(tokens)
    
    return output.last_hidden_state.mean(dim=1).squeeze().numpy()

# Lấy thông tin ngữ cảnh từ các quan hệ
context_query = """
  MATCH (instance:Resource)-[r]->(related:Resource)
  WHERE instance.ns0__maMonHoc IN $codes
  RETURN instance.ns0__maMonHoc AS code, type(r) AS relation, related.rdfs__label AS related_label
"""
context_results = neo4j_graph.run(context_query, codes=[subject[0] for subject in subjects]).data()

# Tạo từ điển ngữ cảnh cho từng môn học
context_dict = {}
for result in context_results:
    code = result['code']
    relation = result['relation']
    related_label = result['related_label']
    
    if code not in context_dict:
        context_dict[code] = []
    context_dict[code].append(f"{relation}: {related_label}")

# Mã hóa các môn học thành vector
subject_vectors = []
for subject in subjects:
    label = subject[1]
    code = subject[0]
    context = ", ".join(context_dict.get(code, []))
    full_description = f"{label}. {context}"
    
    # Gán trọng số cao hơn cho nhãn môn học
    weighted_description = f"{label} " * 3 + full_description
    vector = encode_sentence(weighted_description, phobert_tokenizer, phobert_model)
    subject_vectors.append(vector)

# Sử dụng FAISS để lập chỉ mục và tìm kiếm ngữ nghĩa
dimension = subject_vectors[0].shape[0]
index = faiss.IndexFlatL2(dimension)
subject_vectors_np = np.array(subject_vectors).astype('float32')
index.add(subject_vectors_np)

def semantic_search(query_sentence, top_k=5):
    """
    Tìm kiếm ngữ nghĩa các môn học dựa trên câu truy vấn
    """
    query_vector = encode_sentence(query_sentence, phobert_tokenizer, phobert_model).astype('float32')
    distances, indices = index.search(np.array([query_vector]), top_k)
    
    results = []
    seen_labels = set()
    for i, idx in enumerate(indices[0]):
        label = labels[idx]
        code = subjects[idx][0]
        context = ", ".join(context_dict.get(code, []))
        
        if label not in seen_labels:
            results.append((label, context, distances[0][i]))
            seen_labels.add(label)
    
    return results

# Tìm kiếm thử nghiệm
query = "mạng máy tinh"
search_results = semantic_search(query)

print("Kết quả tìm kiếm ngữ nghĩa:")
for label, context, distance in search_results:
    print(f"Môn học: {label}, Ngữ cảnh: {context}, Khoảng cách: {distance}")


Kết quả tìm kiếm ngữ nghĩa:
Môn học: An toàn thông tin mạng, Ngữ cảnh: ns0__coNoiDung: Nội dung An toàn thông tin mạng, rdf__type: Môn CN An Toàn Thông Tin, rdf__type: None, ns0__hocTruoc: Mạng máy tính, Khoảng cách: 48.58649826049805
Môn học: Mạng máy tính, Ngữ cảnh: ns0__coNoiDung: Nội dung Mạng máy tính, rdf__type: Đại cương ngành, rdf__type: None, ns0__hocTruoc: Nguyên lý hệ điều hành, Khoảng cách: 48.998268127441406
Môn học: Phân tích mã độc, Ngữ cảnh: ns0__coNoiDung: Nội dung Phân tích mã độc, rdf__type: Môn CN An Toàn Thông Tin, rdf__type: None, ns0__hocTruoc: Mạng máy tính, Khoảng cách: 49.40546798706055
Môn học: PBL4: Đồ án hệ điều hành và Mạng máy tính, Ngữ cảnh: ns0__coNoiDung: Nội dung PBL4: Đồ án hệ điều hành và mạng máy tính, rdf__type: Đồ án, rdf__type: None, ns0__songHanh: Lập trình mạng, Khoảng cách: 49.664573669433594
Môn học: Quản trị mạng, Ngữ cảnh: ns0__coNoiDung: Nội dung Quản trị mạng, rdf__type: Môn CN An Toàn Thông Tin, rdf__type: None, ns0__hocTruoc: Mạng máy 